In [ ]:
!pip install pandas
!pip install datasets
!pip install transformers
!pip install sentencepiece
!pip install sacrebleu
!pip install evaluate
!pip install sacrebleu
!pip install accelerate -U
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
folder_path='/content/drive/MyDrive/corpus'
os.listdir(folder_path)

['IITB.en-hi.en', 'IITB.en-hi.hi']

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
folder_path = '/content/drive/MyDrive/corpus'
english_file = '/content/drive/MyDrive/corpus/IITB.en-hi.en'
hindi_file = '/content/drive/MyDrive/corpus/IITB.en-hi.hi'

In [ ]:
with open(english_file, 'r', encoding='utf-8') as f:
    eng_sentences = f.readlines()

with open(hindi_file, 'r', encoding='utf-8') as f:
    hin_sentences = f.readlines()

In [ ]:
hin_sentences = ["<start>" + sentence.strip() + "<end>" for sentence in hin_sentences]

In [ ]:
eng_sentences = eng_sentences[:100]
hin_sentences = hin_sentences[:100]

In [ ]:
eng_sentences

['Give your application an accessibility workout\n',
 'Accerciser Accessibility Explorer\n',
 'The default plugin layout for the bottom panel\n',
 'The default plugin layout for the top panel\n',
 'A list of plugins that are disabled by default\n',
 'Highlight duration\n',
 'The duration of the highlight box when selecting accessible nodes\n',
 'Highlight border color\n',
 'The color and opacity of the highlight border.\n',
 'Highlight fill color\n',
 'The color and opacity of the highlight fill.\n',
 'API Browser\n',
 'Browse the various methods of the current accessible\n',
 'Hide private attributes\n',
 'Method\n',
 'Property\n',
 'Value\n',
 'IPython Console\n',
 'Interactive console for manipulating currently selected accessible\n',
 'Event monitor\n',
 '_ Monitor Events\n',
 'C _ lear Selection\n',
 'Everything\n',
 'Selected application\n',
 'Selected accessible\n',
 'Source\n',
 'Event Monitor\n',
 'Shows events as they occur from selected types and sources\n',
 'Highlight last

In [ ]:
hin_sentences

['<start>अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें<end>',
 '<start>एक्सेर्साइसर पहुंचनीयता अन्वेषक<end>',
 '<start>निचले पटल के लिए डिफोल्ट प्लग-इन खाका<end>',
 '<start>ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका<end>',
 '<start>उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है<end>',
 '<start>अवधि को हाइलाइट रकें<end>',
 '<start>पहुंचनीय आसंधि (नोड) को चुनते समय हाइलाइट बक्से की अवधि<end>',
 '<start>सीमांत (बोर्डर) के रंग को हाइलाइट करें<end>',
 '<start>हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता।<end>',
 '<start>भराई के रंग को हाइलाइट करें<end>',
 '<start>हाइलाइट किया गया भराई का रंग और पारदर्शिता।<end>',
 '<start>एपीआई विचरक<end>',
 '<start>इस समय जिसे प्राप्त किया गया हो, उसकी विभिन्न विधियों (मेथड) में विचरण करें<end>',
 '<start>निजी गुणों को छिपाएं<end>',
 '<start>विधि<end>',
 '<start>गुणधर्म<end>',
 '<start>मान<end>',
 '<start>आईपाइथन कन्सोल<end>',
 '<start>इस समय चुने गए एक्सेसेबेल से काम लेने के लिए अंतर्क्रियात्मक कन्सोल<end>',
 '<start>घटना मानिटर<end>',
 '<st

In [ ]:
eng_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(eng_sentences)
eng_sequences = eng_tokenizer.texts_to_sequences(eng_sentences)
eng_max_len = max(len(seq) for seq in eng_sequences)
eng_sequences = pad_sequences(eng_sequences, maxlen=eng_max_len, padding='post')


In [ ]:
eng_tokenizer.word_index

{'the': 1,
 'of': 2,
 'highlight': 3,
 'accessible': 4,
 'selected': 5,
 'event': 6,
 'color': 7,
 'value': 8,
 'default': 9,
 'plugin': 10,
 'for': 11,
 'and': 12,
 'various': 13,
 'monitor': 14,
 'description': 15,
 'position': 16,
 'application': 17,
 'accessibility': 18,
 'layout': 19,
 'panel': 20,
 'duration': 21,
 'selecting': 22,
 'border': 23,
 'opacity': 24,
 'fill': 25,
 'methods': 26,
 'current': 27,
 'console': 28,
 'events': 29,
 'selection': 30,
 'last': 31,
 'start': 32,
 '0': 33,
 'locale': 34,
 'select': 35,
 'columns': 36,
 'rows': 37,
 'table': 38,
 'name': 39,
 'minimum': 40,
 'interface': 41,
 'inspect': 42,
 'give': 43,
 'your': 44,
 'an': 45,
 'workout': 46,
 'accerciser': 47,
 'explorer': 48,
 'bottom': 49,
 'top': 50,
 'a': 51,
 'list': 52,
 'plugins': 53,
 'that': 54,
 'are': 55,
 'disabled': 56,
 'by': 57,
 'box': 58,
 'when': 59,
 'nodes': 60,
 'api': 61,
 'browser': 62,
 'browse': 63,
 'hide': 64,
 'private': 65,
 'attributes': 66,
 'method': 67,
 'propert

In [ ]:
eng_sequences

array([[ 43,  44,  17,  45,  18,  46,   0,   0,   0,   0],
       [ 47,  18,  48,   0,   0,   0,   0,   0,   0,   0],
       [  1,   9,  10,  19,  11,   1,  49,  20,   0,   0],
       [  1,   9,  10,  19,  11,   1,  50,  20,   0,   0],
       [ 51,  52,   2,  53,  54,  55,  56,  57,   9,   0],
       [  3,  21,   0,   0,   0,   0,   0,   0,   0,   0],
       [  1,  21,   2,   1,   3,  58,  59,  22,   4,  60],
       [  3,  23,   7,   0,   0,   0,   0,   0,   0,   0],
       [  1,   7,  12,  24,   2,   1,   3,  23,   0,   0],
       [  3,  25,   7,   0,   0,   0,   0,   0,   0,   0],
       [  1,   7,  12,  24,   2,   1,   3,  25,   0,   0],
       [ 61,  62,   0,   0,   0,   0,   0,   0,   0,   0],
       [ 63,   1,  13,  26,   2,   1,  27,   4,   0,   0],
       [ 64,  65,  66,   0,   0,   0,   0,   0,   0,   0],
       [ 67,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [ 68,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  8,   0,   0,   0,   0,   0,   0,   0,   0,   0

In [ ]:
eng_max_len

10

In [ ]:
hin_tokenizer = Tokenizer()
hin_tokenizer.fit_on_texts(hin_sentences)
hin_sequences = hin_tokenizer.texts_to_sequences(hin_sentences)
hin_max_len = max(len(seq) for seq in hin_sequences)
hin_sequences = pad_sequences(hin_sequences, maxlen=hin_max_len, padding='post')

In [ ]:
hin_max_len

17

In [ ]:
hin_sequences

array([[  1,  64,  17, ...,   0,   0,   0],
       [  1,  68,  29, ...,   0,   0,   0],
       [  1,  70,  30, ...,   0,   0,   0],
       ...,
       [  1, 198,   2, ...,   0,   0,   0],
       [  1, 199,   2, ...,   0,   0,   0],
       [  1, 200, 201, ...,   0,   0,   0]], dtype=int32)

In [ ]:
hin_tokenizer.word_index

{'start': 1,
 'end': 2,
 'को': 3,
 'के': 4,
 'हाइलाइट': 5,
 'गए': 6,
 'करें': 7,
 'है': 8,
 'मान': 9,
 'चुने': 10,
 'घटना': 11,
 'लिए': 12,
 'प्लग': 13,
 'से': 14,
 'रंग': 15,
 'bgt': 16,
 'अनुप्रयोग': 17,
 'का': 18,
 'डिफोल्ट': 19,
 'इन': 20,
 'किया': 21,
 'गया': 22,
 'समय': 23,
 'और': 24,
 'मानिटर': 25,
 'हैं': 26,
 'स्थिति': 27,
 'स्तंभ': 28,
 'पहुंचनीयता': 29,
 'पटल': 30,
 'खाका': 31,
 'की': 32,
 'अवधि': 33,
 'पहुंचनीय': 34,
 'सीमांत': 35,
 'किए': 36,
 'भराई': 37,
 'इस': 38,
 'विभिन्न': 39,
 'गुणों': 40,
 'कन्सोल': 41,
 'एक्सेसेबेल': 42,
 'चुनाव': 43,
 'सभी': 44,
 'जैसे': 45,
 'अंतिम': 46,
 'करो': 47,
 'नहीं': 48,
 'वर्णन': 49,
 'कार्रवाई': 50,
 '0': 51,
 'स्थान': 52,
 'चुनी': 53,
 'गई': 54,
 'पंक्तियां': 55,
 'lt': 56,
 'सारणी': 57,
 'नाम': 58,
 'x': 59,
 'न्यूनतम': 60,
 'अंतराफलक': 61,
 'एक्सेसेबल': 62,
 'जांचें': 63,
 'अपने': 64,
 'व्यायाम': 65,
 'लाभ': 66,
 'दें': 67,
 'एक्सेर्साइसर': 68,
 'अन्वेषक': 69,
 'निचले': 70,
 'ऊपरी': 71,
 'उन': 72,
 'इनों': 73,
 'सूची': 74,
 'जिन्हें'

In [ ]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
hin_vocab_size = len(hin_tokenizer.word_index) + 1

print(f"English Vocabulary Size: {eng_vocab_size}")
print(f"Hindi Vocabulary Size: {hin_vocab_size}")

English Vocabulary Size: 168
Hindi Vocabulary Size: 202


In [ ]:
encoder_inputs = Input(shape=(eng_max_len,))
encoder_embedding = Embedding(eng_vocab_size, 256)(encoder_inputs)
encoder_lstm = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(hin_vocab_size, 256)(decoder_inputs)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

attention = tf.keras.layers.Attention()([decoder_outputs, encoder_outputs])
decoder_concat_input = tf.keras.layers.Concatenate(axis=-1)([decoder_outputs, attention])

decoder_dense = Dense(hin_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_concat_input)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 10)]                 0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, 10, 256)              43008     ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, None, 256)            51712     ['input_2[0][0]']             
                                                                                              

In [ ]:
hin_target_sequences = np.zeros((len(hin_sequences), hin_max_len, hin_vocab_size), dtype='float32')
for i, seq in enumerate(hin_sequences):
    for t, word_id in enumerate(seq):
        if t > 0:
            hin_target_sequences[i, t - 1, word_id] = 1

decoder_input_sequences = np.array(hin_sequences)[:, :-1]
decoder_target_sequences = hin_target_sequences[:, 1:, :]


In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
hin_target_sequences = hin_sequences[:, 1:]
decoder_input_sequences = np.array(hin_sequences)[:, :-1]
decoder_target_sequences = hin_target_sequences

model.fit([eng_sequences, decoder_input_sequences], decoder_target_sequences,
          batch_size=64,
          epochs=100,
          validation_split=0.2)

Epoch 1/100
2/2 [==============================] - 6s 2s/step - loss: 5.2495 - val_loss: 4.4773
Epoch 2/100
2/2 [==============================] - 0s 250ms/step - loss: 4.2245 - val_loss: 1.7201
Epoch 3/100
2/2 [==============================] - 0s 246ms/step - loss: 1.6729 - val_loss: 1.6092
Epoch 4/100
2/2 [==============================] - 0s 310ms/step - loss: 1.5240 - val_loss: 1.5542
Epoch 5/100
2/2 [==============================] - 0s 259ms/step - loss: 1.4256 - val_loss: 1.4769
Epoch 6/100
2/2 [==============================] - 0s 69ms/step - loss: 1.3537 - val_loss: 1.4537
Epoch 7/100
2/2 [==============================] - 0s 241ms/step - loss: 1.3241 - val_loss: 1.4709
Epoch 8/100
2/2 [==============================] - 0s 57ms/step - loss: 1.2988 - val_loss: 1.5262
Epoch 9/100
2/2 [==============================] - 0s 210ms/step - loss: 1.2936 - val_loss: 1.5241
Epoch 10/100
2/2 [==============================] - 0s 212ms/step - loss: 1.2811 - val_loss: 1.5087
Epoch 11/100
2

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


In [ ]:
hin_tokenizer = Tokenizer(num_words=hin_vocab_size, oov_token='<OOV>')
hin_tokenizer.fit_on_texts(['<start>'] + hin_sentences + ['<end>'])


In [ ]:
import re
def translate(input_text):
    intermediate_representation = {}

    def de_formatter(text):
        text = text.lower()
        text = re.sub(r'[^\w\s]', '', text)
        return text


    def tokenize(text):
        return eng_tokenizer.texts_to_sequences([text])[0]


    def decode_sequence(input_seq):

        states_value = encoder_model.predict(input_seq)


        target_seq = np.zeros((1, 1))


        target_seq[0, 0] = hin_tokenizer.word_index['<start>']


        stop_condition = False
        decoded_sentence = ''
        while not stop_condition:
            output_tokens, h, c = decoder_model.predict([target_seq] + states_value)


            sampled_token_index = np.argmax(output_tokens[0, -1, :])
            sampled_word = hin_tokenizer.index_word[sampled_token_index]

            if sampled_word == '<end>' or len(decoded_sentence) > hin_max_len:
                stop_condition = True
            else:
                decoded_sentence += ' ' + sampled_word


                target_seq = np.zeros((1, 1))
                target_seq[0, 0] = sampled_token_index


                states_value = [h, c]

        return decoded_sentence.strip()

    de_formatted_text = de_formatter(input_text)
    intermediate_representation['de_formatter'] = de_formatted_text

    tokenized_text = tokenize(de_formatted_text)
    intermediate_representation['tokenize'] = tokenized_text

    input_seq = pad_sequences([tokenized_text], maxlen=eng_max_len, padding='post')
    intermediate_representation['encoder_input'] = input_seq

    decoded_sentence = decode_sequence(input_seq)
    intermediate_representation['decoded_sentence'] = decoded_sentence

    return intermediate_representation

input_text = "The quick brown fox jumps over the lazy dog."
intermediate_representation = translate(input_text)

for step, result in intermediate_representation.items():
    print(f"{step}:\n{result}\n")


In [ ]:
def translate_sentence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = hin_tokenizer.word_index['<start>']

    stop_condition = False
    translated_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = hin_tokenizer.index_word[sampled_token_index]

        if sampled_word == '<end>' or len(translated_sentence.split()) > hin_max_len:
            stop_condition = True
        else:
            translated_sentence += ' ' + sampled_word
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return translated_sentence
